In [1]:
from selenium import webdriver
import time

In [2]:
# open chrome browser
browser = webdriver.Chrome(executable_path='../../../chromedriver.exe')

In [3]:
browser.get('https://play.google.com/store/apps/details?id=com.ikozen.ikozenmember&hl=ko-KR')

In [4]:
browser.implicitly_wait(10)

In [5]:
# 리뷰 모두보기 버튼 클릭(팝업 띄우기)
click_path = 'div:nth-child(5) > div > div > button > span'
browser.find_element_by_css_selector(click_path).click()

In [6]:
reviews_bundle = browser.find_elements_by_css_selector('.RHo1pe')
len(reviews_bundle)

40

In [64]:
# 리뷰 한 개 가져오기
review = browser.find_element_by_css_selector('.RHo1pe')
review_writer = review.find_element_by_css_selector('.X5PpBb').text
review_text = review.find_element_by_css_selector('.h3YV2d').text
review_star = review.find_element_by_css_selector('.iXRFPc').get_attribute('aria-label')
print('writer: {}'.format(review_writer))
print('review: {}'.format(review_text))
print('star: {}'.format(review_star))
print('Done')

writer: Google 사용자
review: 입장방식은 변경됐는데 어플은 여전히 바코드만 보여집니다. 짐에서는 더이상 바코드스캔이 아닌 어플에서 입장하기로 바뀌어 사용치 못 하고 있습니다. 매달 이용료는 나가고 있는 상황이면 고쳐지기 전까지 이용못했던 일자에 대해 보상처리는 되나요?어플을 지웠다 다시 다운 받아봐도 똑같은 상태입니다. 도대체 언제부터 전 사용 가능한가요?
star: 별표 5개 만점에 1개를 받았습니다.
Done


In [7]:
import pandas as pd
import pymongo as mg

In [8]:
# 리뷰 모두보기 버튼 클릭(팝업 띄우기)
click_path = 'div:nth-child(5) > div > div > button > span'
browser.find_element_by_css_selector(click_path).click()
reviews_list = list()

In [12]:
review_total_count = 0   # 리뷰 총 갯수
while True:
    reviews_bundle = browser.find_elements_by_css_selector('.RHo1pe')
    review_standard_count_per = len(reviews_bundle)
    if review_total_count == review_standard_count_per:
        break

    review_total_count = review_standard_count_per
    reviews_bundle[len(reviews_bundle)-1].click()
    time.sleep(1)

In [16]:
client = mg.MongoClient(host='mongodb://localhost:27017')
database = client['project_data_analystics']
collection = database['review']

columns_name = ['writer', 'text', 'star']
i = 1
for review in reviews_bundle:
    try:
        review_writer = review.find_element_by_css_selector('div.fysCi > div > div:nth-child(2) > div:nth-child({0}) > header > div.YNR7H > div.gSGphe > div'.format(i)).text
    except:
        review_writer = str()
    try:    
        review_text = review.find_element_by_css_selector('div.fysCi > div > div:nth-child(2) > div:nth-child({0}) > div.h3YV2d'.format(i)).text
    except:
        review_text = str()
    try:
        review_star = review.find_element_by_css_selector('div.fysCi > div > div:nth-child(2) > div:nth-child({0}) > header > div.Jx4nYe > div'.format(i)).get_attribute('aria-label')
    except:
        review_star = str()
    i=i+1
    print(i)
    review_list = [review_writer, review_text, review_star]
    reviews_list.append(review_list)

    # 저장 위한 작업
    df_reviews = pd.DataFrame(data=reviews_list, columns=columns_name)
    data_dict = df_reviews.to_dict(orient='records')
    collection.insert_many(data_dict)

print('Done')

2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
27